In [ ]:
#%pip install crewai==0.119.0 crewai-tools==0.44.0 openai==1.25.1 google-cloud-bigquery==3.17.2 google-auth==2.29.0 google-auth-oauthlib==1.2.0 beautifulsoup4==4.12.3 playwright==1.44.0

In [2]:
%pip install crewai crewai-tools openai google-cloud-bigquery google-auth google-auth-oauthlib beautifulsoup4  playwright anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.1/337.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.0/633.0 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.7 MB/s

In [3]:
import warnings
warnings.filterwarnings('ignore')

from crewai import Agent, Crew, Task
from crewai.tools import BaseTool
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
from google.cloud import bigquery
from google.oauth2 import service_account
import openai
import os
import getpass
from typing import Optional
import anthropic

/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(
/usr/local/lib/python3.11/dist-packages/geopandas/_compat.py:7: DeprecationWarning: The 'shapely.geos' module is deprecated, and will be removed in a future version. All attributes of 'shapely.geos' are available directly from the top-level 'shapely' namespace (since shapely 2.0.0).
  import shapely.geos


In [19]:
# Set up credentials
credentials = service_account.Credentials.from_service_account_file(
    "adsp-34002-on02-prep-sense-58b6ff084106.json"
)
bq_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [5]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [6]:
os.environ["SERPER_API_KEY"] = getpass.getpass("Enter your Serper API key: ")

Enter your Serper API key: ··········


In [7]:
os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")

Enter your Anthropic API key: ··········


In [8]:
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"
openai.api_key = os.environ["OPENAI_API_KEY"]

In [9]:
client = anthropic.Anthropic(
    api_key=os.environ["ANTHROPIC_API_KEY"]
)

In [10]:
# Initialize external tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [11]:
from pydantic import BaseModel, Field

class BigQueryPantryToolInput(BaseModel):
    user_id: int = Field(..., description="The user ID for whom to fetch pantry items")
    pantry_table: Optional[str] = Field(None, description="The BigQuery pantry table")

In [12]:
from typing import Type

class BigQueryPantryTool(BaseTool):
    name: str = "BigQueryPantryTool"
    description: str = "Fetches pantry items from BigQuery"
    args_schema: Type[BaseModel] = BigQueryPantryToolInput

    def _run(self, user_id: int, pantry_table: Optional[str] = None) -> list:
        print(f"[DEBUG] Tool invoked with user_id={user_id}, pantry_table={pantry_table}")
        pantry_table = pantry_table or 'adsp-34002-on02-prep-sense.Inventory.pantry_items'

        query = """
            SELECT
              p.user_id,
              pi.pantry_item_id,
              p.pantry_name,
              pp.product_name,
              pi.quantity,
              pi.used_quantity,
              pi.unit_of_measurement,
              pi.expiration_date,
              pi.created_at
            FROM `adsp-34002-on02-prep-sense.Inventory.pantry_items` AS pi
            JOIN `adsp-34002-on02-prep-sense.Inventory.pantry` AS p
              ON pi.pantry_id = p.pantry_id
            JOIN `adsp-34002-on02-prep-sense.Inventory.products` AS pp
              ON pi.pantry_item_id = pp.pantry_item_id
            WHERE p.user_id = @user_id
              AND pi.quantity > pi.used_quantity
            order by pantry_item_id
            limit 50;
        """

        job_config = bigquery.QueryJobConfig(
            query_parameters=[
                bigquery.ScalarQueryParameter("user_id", "INT64", user_id)
            ]
        )

        try:
            query_job = bq_client.query(query, job_config=job_config)
            results = query_job.result()
            return [dict(row) for row in results]
        except Exception as e:
            return [{"error": str(e)}]

In [13]:
# testing for a specific user

bigquery_tool = BigQueryPantryTool()
results = bigquery_tool._run(user_id=111)
print(results)

[DEBUG] Tool invoked with user_id=111, pantry_table=None
[{'user_id': 111, 'pantry_item_id': 5001, 'pantry_name': 'Samantha’s Pantry', 'product_name': 'Milk', 'quantity': 5.0, 'used_quantity': 1, 'unit_of_measurement': 'gal', 'expiration_date': datetime.date(2026, 6, 26), 'created_at': datetime.datetime(2025, 5, 28, 2, 0)}, {'user_id': 111, 'pantry_item_id': 5002, 'pantry_name': 'Samantha’s Pantry', 'product_name': 'Chicken Breast', 'quantity': 2.0, 'used_quantity': 0, 'unit_of_measurement': 'lb', 'expiration_date': datetime.date(2025, 7, 27), 'created_at': datetime.datetime(2025, 5, 28, 2, 0)}, {'user_id': 111, 'pantry_item_id': 5005, 'pantry_name': 'Samantha’s Pantry', 'product_name': 'Extra-Virgin Olive Oil', 'quantity': 1.0, 'used_quantity': 0, 'unit_of_measurement': 'L', 'expiration_date': datetime.date(2027, 5, 28), 'created_at': datetime.datetime(2025, 5, 28, 2, 0)}, {'user_id': 111, 'pantry_item_id': 5006, 'pantry_name': 'Samantha’s Pantry', 'product_name': 'Black Beans – 15 oz

In [14]:
## Ingredient filter class

bq_client = bigquery.Client()

class IngredientFilterTool(BaseTool):
    name: str = "IngredientFilterTool"
    description: str = "Fetches non-expired and available pantry items for a given user from BigQuery"
    args_schema: Type[BaseModel] = BigQueryPantryToolInput

    def _run(self, user_id: int, pantry_table: Optional[str] = None) -> list:
        print(f"[DEBUG] Tool invoked with user_id={user_id}, pantry_table={pantry_table}")
        pantry_table = pantry_table or 'adsp-34002-on02-prep-sense.Inventory.pantry_items'

        query = """
            SELECT
              p.user_id,
              pi.pantry_item_id,
              p.pantry_name,
              pp.product_name,
              pi.quantity,
              pi.used_quantity,
              pi.unit_of_measurement,
              pi.expiration_date,
              pi.created_at
            FROM `adsp-34002-on02-prep-sense.Inventory.pantry_items` AS pi
            JOIN `adsp-34002-on02-prep-sense.Inventory.pantry` AS p
              ON pi.pantry_id = p.pantry_id
            JOIN `adsp-34002-on02-prep-sense.Inventory.products` AS pp
              ON pi.pantry_item_id = pp.pantry_item_id
            WHERE p.user_id = @user_id
              AND pi.quantity > pi.used_quantity
              AND pi.expiration_date >= CURRENT_DATE()
              AND pi.quantity > 0
            order by pantry_item_id
            limit 50;
        """

        job_config = bigquery.QueryJobConfig(
            query_parameters=[
                bigquery.ScalarQueryParameter("user_id", "INT64", user_id)
            ]
        )

        try:
            query_job = bq_client.query(query, job_config=job_config)
            results = query_job.result()
            return [dict(row) for row in results]
        except Exception as e:
            return [{"error": str(e)}]

In [17]:
ingredient_tool = IngredientFilterTool()
results = ingredient_tool._run(user_id=111)
print(results)

[DEBUG] Tool invoked with user_id=111, pantry_table=None
[{'user_id': 111, 'pantry_item_id': 5001, 'pantry_name': 'Samantha’s Pantry', 'product_name': 'Milk', 'quantity': 5.0, 'used_quantity': 1, 'unit_of_measurement': 'gal', 'expiration_date': datetime.date(2026, 6, 26), 'created_at': datetime.datetime(2025, 5, 28, 2, 0)}, {'user_id': 111, 'pantry_item_id': 5002, 'pantry_name': 'Samantha’s Pantry', 'product_name': 'Chicken Breast', 'quantity': 2.0, 'used_quantity': 0, 'unit_of_measurement': 'lb', 'expiration_date': datetime.date(2025, 7, 27), 'created_at': datetime.datetime(2025, 5, 28, 2, 0)}, {'user_id': 111, 'pantry_item_id': 5005, 'pantry_name': 'Samantha’s Pantry', 'product_name': 'Extra-Virgin Olive Oil', 'quantity': 1.0, 'used_quantity': 0, 'unit_of_measurement': 'L', 'expiration_date': datetime.date(2027, 5, 28), 'created_at': datetime.datetime(2025, 5, 28, 2, 0)}, {'user_id': 111, 'pantry_item_id': 5006, 'pantry_name': 'Samantha’s Pantry', 'product_name': 'Black Beans – 15 oz

In [18]:
class UserRestrictionToolInput(BaseModel):
    user_id: int

class UserRestrictionTool(BaseTool):
    name: str = "UserRestrictionTool"
    description: str = "Fetches dietary restrictions and allergens for a user"
    args_schema: Type[BaseModel] = UserRestrictionToolInput

    def _run(self, user_id: int) -> dict:
        query = """
            SELECT dietary_preference, allergens
            FROM `adsp-34002-on02-prep-sense.Inventory.user_preference`
            WHERE user_id = @user_id
        """

        job_config = bigquery.QueryJobConfig(
            query_parameters=[
                bigquery.ScalarQueryParameter("user_id", "INT64", user_id)
            ]
        )

        try:
            query_job = bq_client.query(query, job_config=job_config)
            result = list(query_job.result())
            if result:
                return dict(result[0])
            else:
                return {"message": "No dietary data found for this user."}
        except Exception as e:
            return {"error": str(e)}

In [20]:
user_restriction_tool = UserRestrictionTool()
results = user_restriction_tool._run(user_id=111)
print(results)

{'dietary_preference': ['vegetarian'], 'allergens': ['peanuts']}


In [21]:

def evaluate_recipe_with_claude(recipe_text, critique_goal="taste, clarity, feasibility, and ingredient alignment"):
    prompt = f"""You are a food critic. Your job is to evaluate the following recipe based on {critique_goal}.

Recipe:
{recipe_text}

Please return a brief critique and a score from 1 to 5.
"""

    response = client.messages.create(
        model="claude-opus-4-20250514",
        max_tokens=512,
        temperature=0.5,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    return response.content[0].text.strip()


In [22]:
class ClaudeRecipeEvaluatorTool(BaseTool):
    name: str = "ClaudeRecipeEvaluatorTool"
    description: str = "Evaluates a recipe for taste, clarity, feasibility, and ingredient alignment using Claude AI."

    def _run(self, recipe_text: str) -> str:
        return evaluate_recipe_with_claude(recipe_text)


In [23]:
recipe_evaluator_tool = ClaudeRecipeEvaluatorTool()

In [24]:
# Define AI Agents for Pantry-to-Dinner Workflow
pantry_scan_agent = Agent(
    role="Pantry Scan Agent",
    goal="Retrieve list of available ingredients for user ID {user_id} from BigQuery pantry table",
    tools=[bigquery_tool],
    verbose=True,
    backstory="You have access to structured data sources like BigQuery and can extract relevant pantry contents."
)

ingredient_filter_agent = Agent(
    role="Ingredient Filter Agent",
    goal="Filter out expired or unusable items from the pantry list",
    tools=[ingredient_tool],
    verbose=True,
    backstory="You ensure only safe and usable ingredients are passed to recipe generation agents."
)

recipe_search_agent = Agent(
    role="Recipe Search Agent",
    goal="Find recipes that can be made using the filtered ingredients",
    tools=[search_tool, scrape_tool, ingredient_tool],
    verbose=True,
    backstory="You search online and through structured recipe repositories to find viable dinner options."
)

nutritional_agent = Agent(
    role="Nutritional Agent",
    goal="Evaluate the nutritional balance of each proposed recipe",
    tools=[search_tool],
    verbose=True,
    backstory="You ensure that dinner suggestions are healthy and balanced. Give me a detailed nutritional breakdown."
)

user_preferences_agent = Agent(
    role="User Preferences Agent",
    goal="Filter out ingredients and recipes that conflict with the user's dietary restrictions and allergens",
    tools=[user_restriction_tool],
    verbose=True,
    backstory="You ensure the user only sees recipes and ingredients that comply with their health and dietary requirements."
)

recipe_scoring_agent = Agent(
    role="Recipe Scoring Agent",
    goal="Score recipes based on match with ingredients, nutrition, and user preferences",
    tools=[],
    verbose=True,
    backstory="You rank suggestions to make it easier for users to choose."
)

response_formatting_agent = Agent(
    role="Response Formatting Agent",
    goal="Format and summarize the best recipe suggestions for user-friendly viewing",
    tools=[],
    verbose=True,
    backstory="You turn results into clean summaries, including ingredients and nutrition info."
)

recipe_evaluator_agent = Agent(
    role="Recipe Evaluator",
    goal="Evaluate the quality and correctness of generated recipes.",
    tools=[recipe_evaluator_tool],
    verbose=True,
    backstory="An expert food critic with access to Claude AI to help rate and review recipes.",
    allow_delegation=False
)

In [25]:
# Define Tasks for Agents

def create_task(agent, description, expected_output, input_variables=None):
    print(f"[DEBUG] Creating task for {agent.role} with inputs: {input_variables}")
    return Task(
        description=description,
        expected_output=expected_output,
        human_input=False,
        agent=agent,
        input_variables=input_variables or []
    )

In [30]:
tasks = [
    create_task(
        pantry_scan_agent,
        "Query BigQuery table for usable ingredients for a given user_id",
        "List of pantry ingredients for a given user_id",
        input_variables=["user_id", "pantry_table"]
    ),
        create_task(
        ingredient_filter_agent,
        "Filter out expired or unusable items from the user's pantry list based on expiration date and usage",
        "List of usable (non-expired) pantry ingredients for a given user_id",
        input_variables=["user_id", "pantry_table"]
    ),
    create_task(recipe_search_agent, "Find dinner recipes that use these ingredients", "List of at least 5 suitable recipes"),
    create_task(nutritional_agent, "Evaluate each recipe for nutritional value", "Nutritional score and comments for each recipe"),
    create_task(
        user_preferences_agent,
        "Filter recipes based on user dietary preferences",
        "List of safe ingredients or recipes based on user dietary restrictions",
        input_variables=["user_id"]
    ),
    create_task(recipe_scoring_agent, "Score recipes on match, nutrition, and user preference", "Ranked list of recipes"),
    create_task(recipe_evaluator_agent, "Evaluate the quality and correctness for each of the top 3 generated recipes.", "A short critique of the recipe including a rating from 1 to 5 and specific feedback.", input_variables=["recipe_text"]),
    create_task(response_formatting_agent, "Format top 3 recipes with ingredients, instructions, nutrition summary, and critiques", "Formatted response in markdown")
]

[DEBUG] Creating task for Pantry Scan Agent with inputs: ['user_id', 'pantry_table']

[DEBUG] Creating task for Ingredient Filter Agent with inputs: ['user_id', 'pantry_table']

[DEBUG] Creating task for Recipe Search Agent with inputs: None

[DEBUG] Creating task for Nutritional Agent with inputs: None

[DEBUG] Creating task for User Preferences Agent with inputs: ['user_id']

[DEBUG] Creating task for Recipe Scoring Agent with inputs: None

[DEBUG] Creating task for Recipe Evaluator with inputs: ['recipe_text']

[DEBUG] Creating task for Response Formatting Agent with inputs: None

In [31]:
# Create and run the Crew
pantry_dinner_crew = Crew(
    agents=[
        pantry_scan_agent,
        ingredient_filter_agent,
        recipe_search_agent,
        nutritional_agent,
        user_preferences_agent,
        recipe_scoring_agent,
        recipe_evaluator_agent,
        response_formatting_agent
    ],
    tasks=tasks,
    verbose=True
)

inputs = {
 #   "user_dietary_preferences": "vegetarian, no nuts",
    "pantry_table": "adsp-34002-on02-prep-sense.Inventory.pantry_items",
    "user_id":111
}

In [32]:
print("[DEBUG] Final Inputs to Crew:", inputs)
result = pantry_dinner_crew.kickoff(inputs=inputs)
print(result)

[DEBUG] Final Inputs to Crew: {'pantry_table': 'adsp-34002-on02-prep-sense.Inventory.pantry_items', 'user_id': 111}

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4810e3b5-38d7-4b2d-b448-7d7df1fa37c8                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Pantry Scan Agent                                                                                       │
│                                                                                                                 │
│  Task: Query BigQuery table for usable ingredients for a given user_id                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

[DEBUG] Tool invoked with user_id=111, pantry_table=pantry_table

Output()

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Response Formatting Agent                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  ## Recipe 1: Grilled Chicken, Charred Corn, and Broccoli Salad                                                 │
│  **URL:** [Grilled Chicken, Charred Corn and Broccoli                                                           │
│  Salad](https://ournourishingtable.com/corn-and-broccoli-salad/)                                                │
│                                                                                                                 │
│  ### Ingredients:                                                                                               │
│  - Chicken Breast: 2 lbs                                                                                        │
│  - Corn on the Cob: 2 units                                                                                     │
│  - Broccoli: 1 unit                                                                                             │
│  - Extra-Virgin Olive Oil: 2 tablespoons                                                                        │
│  - Salt and Pepper to taste                                                                                     │
│                                                                                                                 │
│  ### Instructions:                                                                                              │
│  1. Prep the chicken by marinating it with olive oil, salt, and pepper.                                         │
│  2. Grill the chicken on medium heat until cooked through.                                                      │
│  3. Meanwhile, char the corn on the grill until it gets a nice color.                                           │
│  4. Chop the broccoli into bite-size pieces and blanch briefly in boiling water.                                │
│  5. Combine grilled chicken, charred corn kernels, and broccoli in a bowl.                                      │
│  6. Serve salad warm or at room temperature.                                                                    │
│                                                                                                                 │
│  ### Nutrition Summary:                                                                                         │
│  - **Protein:** High due to chicken breast                                                                      │
│  - **Fiber & Carbs:** Moderate contribution from broccoli and corn                                              │
│                                                                                                                 │
│  ### Critique:                                                                                                  │
│  - **Score:** 4/5                                                                                               │
│  - The recipe is a nutritious option with a great balance of flavors. It's easy to prepare, though it could     │
│  benefit from a more defined dressing to enhance flavor cohesion.                                               │
│                                                                                                                 │
│  ---                                                   

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: bb66d08c-fa6b-4fe3-a9dc-4506072cd411                                                                     │
│  Agent: Response Formatting Agent                                                                               │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 4810e3b5-38d7-4b2d-b448-7d7df1fa37c8                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```markdown                                                                                      │
│  ## Recipe 1: Grilled Chicken, Charred Corn, and Broccoli Salad                                                 │
│  **URL:** [Grilled Chicken, Charred Corn and Broccoli                                                           │
│  Salad](https://ournourishingtable.com/corn-and-broccoli-salad/)                                                │
│                                                                                                                 │
│  ### Ingredients:                                                                                               │
│  - Chicken Breast: 2 lbs                                                                                        │
│  - Corn on the Cob: 2 units                                                                                     │
│  - Broccoli: 1 unit                                                                                             │
│  - Extra-Virgin Olive Oil: 2 tablespoons                                                                        │
│  - Salt and Pepper to taste                                                                                     │
│                                                                                                                 │
│  ### Instructions:                                                                                              │
│  1. Prep the chicken by marinating it with olive oil, salt, and pepper.                                         │
│  2. Grill the chicken on medium heat until cooked through.                                                      │
│  3. Meanwhile, char the corn on the grill until it gets a nice color.                                           │
│  4. Chop the broccoli into bite-size pieces and blanch briefly in boiling water.                                │
│  5. Combine grilled chicken, charred corn kernels, and broccoli in a bowl.                                      │
│  6. Serve salad warm or at room temperature.                                                                    │
│                                                                                                                 │
│  ### Nutrition Summary:                                                                                         │
│  - **Protein:** High due to chicken breast                                                                      │
│  - **Fiber & Carbs:** Moderate contribution from broccoli and corn                                              │
│                                                                                                                 │
│  ### Critique:                                                                                                  │
│  - **Score:** 4/5                                                                                               │
│  - The recipe is a nutritious option with a great balance of flavors. It's easy to prepare, though it could     │
│  benefit from a more defined dressing to enhance flavor cohesion.                                               │
│                                                       

```markdown
## Recipe 1: Grilled Chicken, Charred Corn, and Broccoli Salad
**URL:** [Grilled Chicken, Charred Corn and Broccoli Salad](https://ournourishingtable.com/corn-and-broccoli-salad/)

### Ingredients:
- Chicken Breast: 2 lbs
- Corn on the Cob: 2 units
- Broccoli: 1 unit
- Extra-Virgin Olive Oil: 2 tablespoons
- Salt and Pepper to taste

### Instructions:
1. Prep the chicken by marinating it with olive oil, salt, and pepper.
2. Grill the chicken on medium heat until cooked through.
3. Meanwhile, char the corn on the grill until it gets a nice color.
4. Chop the broccoli into bite-size pieces and blanch briefly in boiling water.
5. Combine grilled chicken, charred corn kernels, and broccoli in a bowl.
6. Serve salad warm or at room temperature.

### Nutrition Summary:
- **Protein:** High due to chicken breast
- **Fiber & Carbs:** Moderate contribution from broccoli and corn

### Critique:
- **Score:** 4/5
- The recipe is a nutritious option with a great balance of flavors. It's 